In [8]:
from transformers import AutoModelForMaskedLM, AutoModelForSequenceClassification
import torch
from datasets import load_dataset
import math
from torch.utils.data import DataLoader
from transformers import default_data_collator
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import preprocessing
import argparse
import preprocessing
import pickle
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn

/home/laurinemeier/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
model_checkpoint = "KBLab/bert-base-swedish-cased"
model =  AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at KBLab/bert-base-swedish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_finetuned = AutoModelForSequenceClassification.from_pretrained("finetuning_hugging_python-finetuned-imdb/checkpoint-920384")
model_finetuned=model_finetuned.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at finetuning_hugging_python-finetuned-imdb/checkpoint-920384 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
data_files = {"train": "swerick_data_party_train.pkl", "test": "swerick_data_party_test.pkl"}
party_dataset = load_dataset("pandas",data_files=data_files)
print(party_dataset)

Generating train split: 3378877 examples [00:18, 179532.30 examples/s]
Generating test split: 725974 examples [00:03, 205408.72 examples/s]

DatasetDict({
    train: Dataset({
        features: ['protocole', 'Note', 'id', 'party', 'gender'],
        num_rows: 3378877
    })
    test: Dataset({
        features: ['protocole', 'Note', 'id', 'party', 'gender'],
        num_rows: 725974
    })
})


In [ ]:
party_dataset["train"][10]

{'protocole': 'data/1867/prot-1867--ak--0119.xml',
 'Note': 'För min del kan jag ej se något hinder för företagande af dessa\n            val ännu något tidigare än Herr Talmannen föreslagit. Jag har\n            föreställt mig, att vi så mycket som möjligt borde taga vara på\n            den dyrbara tiden, och att nu ifrågakomna val kunde förrättas\n            t. ex. på Tisdagen, och valen till suppleanter i Utskotten på\n            Thorsdagen.',
 'id': 'i-PyY1Vo1W6WaajphhpnKHN8',
 'party': None,
 'gender': 'man'}

Classification for parties

In [13]:
def filter_Nan(subset,example):
    return example[subset] is not None


party_dataset['train']=party_dataset['train'].filter(lambda x: filter_Nan('party',x))
party_dataset['test']=party_dataset['test'].filter(lambda x: filter_Nan('party',x))



Filter: 100%|██████████| 725974/725974 [00:04<00:00, 178776.66 examples/s]


In [17]:
label_encoder = LabelEncoder()
label_encoder.fit(party_dataset['train']['party'])
party_dataset['train'] = party_dataset['train'].map(lambda example: {'party_label': label_encoder.transform([example['party']])[0]}, 
                                                    remove_columns=['party'])
party_dataset['test'] = party_dataset['test'].map(lambda example: {'party_label': label_encoder.transform([example['party']])[0]}, 
                                                    remove_columns=['party'])

Map: 100%|██████████| 676215/676215 [01:50<00:00, 6125.07 examples/s]


AttributeError: 'Dataset' object has no attribute 'head'

In [ ]:
label_encoder

In [1]:
party_dataset.features

NameError: name 'party_dataset' is not defined

In [ ]:
party_dataset=party_dataset.remove_columns("gender","id")

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def tokenize_function(example,subset):
    return tokenizer(example[subset], truncation=True,
                    add_special_tokens = True,
                    max_length = 512,
                    padding = 'max_length',
                    return_attention_mask = True,)


In [2]:
party_tokenized = party_dataset.map(lambda x:tokenize_function(x,"Note"))


batch_size=64
num_workers=4
train_loader = DataLoader(
            party_dataset["train],
            shuffle=True,
            batch_size = batch_size,
            num_workers = num_workers
        )

test_loader = DataLoader(
            party_dataset["test],
            shuffle=False,
            batch_size = batch_size,
            num_workers = num_workers
        )

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
        "finetuning_hugging_python-finetuned-imdb/checkpoint-920384",
        num_labels=len(label_encoder),
        id2label=label_dict).to(device)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()), lr=args.learning_rate)
num_training_steps = len(train_loader) * args.n_epochs
num_warmup_steps = num_training_steps // 10


In [ ]:
 # Linear warmup and step decay
    scheduler = get_linear_schedule_with_warmup(
        optimizer = optimizer,
        num_warmup_steps = num_warmup_steps,
        num_training_steps = num_training_steps
        )


In [ ]:
def evaluate(model, loader):
    loss, accuracy = 0.0, []
    model.eval()
    for batch in tqdm(loader, total=len(loader)):
        input_ids = batch[0].to(args.device)
        input_mask = batch[1].to(args.device)
        labels = batch[2].to(args.device)
        output = model(input_ids,
            token_type_ids=None, 
            attention_mask=input_mask, 
            labels=labels)
        loss += output.loss.item()
        preds_batch = torch.argmax(output.logits, axis=1)
        batch_acc = torch.mean((preds_batch == labels).float())
        accuracy.append(batch_acc)
        
    accuracy = torch.mean(torch.tensor(accuracy))
    return loss, accuracy

In [ ]:
n_epoch=10
train_losses=[]
valid_losses=[]
for epoch in range(n_epochs):
    print(f"Epoch {epoch} starts!")
    train_loss = 0
    model.train()
    for batch in tqdm(train_loader, total=len(train_loader)):
        model.zero_grad()   

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)
        output = model(input_ids,
            token_type_ids=None, 
            attention_mask=input_mask, 
            labels=labels)
        loss = output.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()
        
    valid_loss, valid_accuracy = evaluate(model, valid_loader)

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    train_loss_avg = train_loss * args.batch_size / len(train_loader)
    valid_loss_avg = valid_loss * args.batch_size / len(valid_loader)

    print(f'Training Loss: {train_loss_avg:.3f}')
    print(f'Validation Loss: {valid_loss_avg:.3f}')
    print(f'Validation accuracy: {valid_accuracy}')
        

In [ ]:
def compare_model():
    loss1, accuracy1 = 0.0, []
    loss2, accuracy2 = 0.0, []
    model.eval()
    model_hugging_face.eval()
    true_labels, pred_labels1, pred_labels2 = [], [], []
    for batch in tqdm(loader, total=len(loader)):
        input_ids = batch[0].to(args.device)
        input_mask = batch[1].to(args.device)
        labels = batch[2].to(args.device)
        output1 = model1(input_ids, token_type_ids=None, attention_mask=input_mask, labels=labels)
        output2 = model2(input_ids, token_type_ids=None, attention_mask=input_mask, labels=labels)
        loss1 += output1.loss.item()
        loss2 += output2.loss.item()
        preds_batch1 = torch.argmax(output1.logits, axis=1)
        preds_batch2 = torch.argmax(output2.logits, axis=1)
        batch_acc1 = torch.mean((preds_batch1 == labels).float())
        batch_acc2 = torch.mean((preds_batch2 == labels).float())
        accuracy1.append(batch_acc1)
        accuracy2.append(batch_acc2)
        true_labels.extend(labels.cpu().numpy())
        pred_labels1.extend(preds_batch1.cpu().numpy())
        pred_labels2.extend(preds_batch2.cpu().numpy())
            
            for true_label, pred_label1, pred_label2 , input_id  in zip(labels, preds_batch1, preds_batch2, input_ids):
                if true_label != pred_label1 or true_label != pred_label2 :
                    text = tokenizer.decode(input_id, skip_special_tokens=True)
                    matching_rows = dataset[dataset['content'].apply(lambda x: Levenshtein.ratio(text, x) >= 0.9)]
                    if not matching_rows.empty:
                        github = matching_rows['github'].iloc[0]
                        protocol_id = matching_rows['protocol_id'].iloc[0]

                        misclassified_examples.append({'text': text, 'true_label': label_names[true_label.item()], 'predicted1':label_names[pred_label1.item()],'predicted2':label_names[pred_label2.item()], 'github': github, 'protocol_id': protocol_id})
                    else:
                        print(f"no matching row for text: {text}")

        misclassified_df = pd.DataFrame(misclassified_examples)
        misclassified_df.to_csv('data/compare_misclassified_examples.csv', index=False, columns=['text', 'true_label', 'predicted1', 'predicted2', 'github', 'protocol_id'])
        
    # Print misclassified examples
        print("\nMisclassified Examples:")
        print(misclassified_examples)

        print("\nAccuracy model 1:", accuracy_score(true_labels, pred_labels1))
        print("\nClassification Report:")
        print(classification_report(true_labels, pred_labels1, target_names=list(label_names)))

        
        print("\nAccuracy model 2:", accuracy_score(true_labels, pred_labels2))
        print("\nClassification Report:")
        print(classification_report(true_labels, pred_labels2, target_names=list(label_names)))